In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [ ]:
df_col=['user_id', 'transaction_amount', 'payment_method', 'date','success/failure']
offers_df_col=['payment_method', 'start_date', 'end_date', 'cashbacks', 'charges']

In [ ]:
import mysql.connector

mydb=mysql.connector.connect(
    host="localhost",
    user="root",
    password="password",
    database="hackonamazon"
)

mycursor=mydb.cursor()
mycursor.execute("SELECT * FROM transactions")
myresult=mycursor.fetchall()

df = pd.DataFrame(columns=df_col)

for x in myresult:
    new_data = pd.DataFrame([x],columns=df_col)
    df=df.append(new_data,ignore_index=True)
    
mycursor.execute("SELECT * FROM offers")
myresult=mycursor.fetchall()

offers_df = pd.DataFrame(columns=offers_df_col)

for x in myresult:
    new_data = pd.DataFrame([x],columns=offers_df_col)
    df=df.append(new_data,ignore_index=True)


In [4]:
df.head()

,user_id,transaction_amount,payment_method,date,success/failure
0,1,150.25,credit_or_debit(visa),2024-06-01,success
1,2,300.50,credit_or_debit(mastercard),2024-06-02,failure
2,1,50.75,net_banking(icici_bank),2024-06-03,success
3,3,200.00,net_banking(kotak_bank),2024-06-04,success
4,4,75.99,credit_or_debit(rupay),2024-06-05,failure


In [571]:
payment_method_counts = df.groupby(['user_id', 'payment_method']).size().reset_index(name='count')
payment_method_counts

,user_id,payment_method,count
0,1,EMI,1
1,1,cash_on_delivery,5
2,1,credit_or_debit(maestro),5
3,1,credit_or_debit(rupay),1
4,1,credit_or_debit(visa),4
5,1,net_banking(icici_bank),1
6,1,net_banking(kotak_bank),3
7,2,EMI,1
8,2,credit_or_debit(mastercard),2
9,2,credit_or_debit(visa),1


In [572]:
most_frequent_payment_method = payment_method_counts.loc[payment_method_counts.groupby('user_id')['count'].idxmax()]
most_frequent_payment_method

,user_id,payment_method,count
1,1,cash_on_delivery,5
11,2,net_banking(state_bank_of_india),4
13,3,net_banking(axis_bank),4
16,4,credit_or_debit(american express),6
23,5,net_banking(hdfc_bank),4
24,6,EMI,3
27,7,credit_or_debit(visa),4
28,8,credit_or_debit(mastercard),4
30,9,credit_or_debit(rupay),3
31,10,credit_or_debit(maestro),3


In [573]:
user_payment_map = most_frequent_payment_method.set_index('user_id')['payment_method'].to_dict()
user_payment_map

{1: 'cash_on_delivery',
 2: 'net_banking(state_bank_of_india)',
 3: 'net_banking(axis_bank)',
 4: 'credit_or_debit(american express)',
 5: 'net_banking(hdfc_bank)',
 6: 'EMI',
 7: 'credit_or_debit(visa)',
 8: 'credit_or_debit(mastercard)',
 9: 'credit_or_debit(rupay)',
 10: 'credit_or_debit(maestro)',
 11: 'credit_or_debit(mastercard)',
 12: 'credit_or_debit(rupay)'}

In [574]:
df['frequently_used_payment_method'] = df['user_id'].map(user_payment_map)
transaction_df=df
transaction_df.head()

,user_id,transaction_amount,payment_method,date,success/failure,frequently_used_payment_method
0,1,150.25,credit_or_debit(visa),2024-06-01,success,cash_on_delivery
1,2,300.50,credit_or_debit(mastercard),2024-06-02,failure,net_banking(state_bank_of_india)
2,1,50.75,net_banking(icici_bank),2024-06-03,success,cash_on_delivery
3,3,200.00,net_banking(kotak_bank),2024-06-04,success,net_banking(axis_bank)
4,4,75.99,credit_or_debit(rupay),2024-06-05,failure,credit_or_debit(american express)


In [575]:
transaction_df['date'] = pd.to_datetime(transaction_df['date'])
transaction_df

,user_id,transaction_amount,payment_method,date,success/failure,frequently_used_payment_method
0,1,150.25,credit_or_debit(visa),2024-06-01,success,cash_on_delivery
1,2,300.50,credit_or_debit(mastercard),2024-06-02,failure,net_banking(state_bank_of_india)
2,1,50.75,net_banking(icici_bank),2024-06-03,success,cash_on_delivery
3,3,200.00,net_banking(kotak_bank),2024-06-04,success,net_banking(axis_bank)
4,4,75.99,credit_or_debit(rupay),2024-06-05,failure,credit_or_debit(american express)
...,...,...,...,...,...,...
82,4,230.50,credit_or_debit(american express),2024-08-22,success,credit_or_debit(american express)
83,12,95.99,credit_or_debit(rupay),2024-08-23,success,credit_or_debit(rupay)
84,5,240.25,net_banking(hdfc_bank),2024-08-24,failure,net_banking(hdfc_bank)
85,1,180.50,credit_or_debit(maestro),2024-08-25,success,cash_on_delivery


In [7]:
offers_df.head()

,payment_method,start_date,end_date,cashbacks,charges
0,credit_or_debit(visa),2024-01-01,2024-12-31,5.0,2.0
1,credit_or_debit(mastercard),2024-01-01,2024-12-31,4.5,2.2
2,credit_or_debit(rupay),2024-01-01,2024-12-31,4.0,1.8
3,credit_or_debit(american express),2024-01-01,2024-12-31,6.0,2.5
4,credit_or_debit(maestro),2024-01-01,2024-12-31,3.5,1.7


In [577]:
offers_df['start_date'] = pd.to_datetime(offers_df['start_date'])
offers_df['end_date'] = pd.to_datetime(offers_df['end_date'])
offers_df

,payment_method,start_date,end_date,cashbacks,charges
0,credit_or_debit(visa),2024-01-01,2024-12-31,5.0,2.0
1,credit_or_debit(mastercard),2024-01-01,2024-12-31,4.5,2.2
2,credit_or_debit(rupay),2024-01-01,2024-12-31,4.0,1.8
3,credit_or_debit(american express),2024-01-01,2024-12-31,6.0,2.5
4,credit_or_debit(maestro),2024-01-01,2024-12-31,3.5,1.7
5,net_banking(axis_bank),2024-01-01,2024-12-31,7.0,1.5
6,net_banking(hdfc_bank),2024-01-01,2024-12-31,6.5,1.8
7,net_banking(kotak_bank),2024-01-01,2024-12-31,7.5,1.9
8,net_banking(icici_bank),2024-01-01,2024-12-31,6.0,1.6
9,net_banking(state_bank_of_india),2024-01-01,2024-12-31,8.0,2.0


In [578]:
merged_df = transaction_df.merge(offers_df, on='payment_method')
merged_df = merged_df[(merged_df['date'] >= merged_df['start_date']) & (merged_df['date'] <= merged_df['end_date'])]
merged_df=merged_df.drop(columns=['start_date','end_date'])
merged_df

,user_id,transaction_amount,payment_method,date,success/failure,frequently_used_payment_method,cashbacks,charges
0,1,150.25,credit_or_debit(visa),2024-06-01,success,cash_on_delivery,5.0,2.0
1,2,300.50,credit_or_debit(mastercard),2024-06-02,failure,net_banking(state_bank_of_india),4.5,2.2
2,1,50.75,net_banking(icici_bank),2024-06-03,success,cash_on_delivery,6.0,1.6
3,3,200.00,net_banking(kotak_bank),2024-06-04,success,net_banking(axis_bank),7.5,1.9
4,4,75.99,credit_or_debit(rupay),2024-06-05,failure,credit_or_debit(american express),4.0,1.8
...,...,...,...,...,...,...,...,...
82,4,230.50,credit_or_debit(american express),2024-08-22,success,credit_or_debit(american express),6.0,2.5
83,12,95.99,credit_or_debit(rupay),2024-08-23,success,credit_or_debit(rupay),4.0,1.8
84,5,240.25,net_banking(hdfc_bank),2024-08-24,failure,net_banking(hdfc_bank),6.5,1.8
85,1,180.50,credit_or_debit(maestro),2024-08-25,success,cash_on_delivery,3.5,1.7


In [579]:
le = LabelEncoder()
merged_df['net_benefit'] = merged_df['cashbacks'] - merged_df['charges']
merged_df['success/failure']=le.fit_transform(merged_df['success/failure'])
merged_df['success_rate'] = merged_df.groupby(['payment_method'])['success/failure'].transform(lambda x: x.sum() / x.count())
merged_df

,user_id,transaction_amount,payment_method,date,success/failure,frequently_used_payment_method,cashbacks,charges,net_benefit,success_rate
0,1,150.25,credit_or_debit(visa),2024-06-01,1,cash_on_delivery,5.0,2.0,3.0,0.777778
1,2,300.50,credit_or_debit(mastercard),2024-06-02,0,net_banking(state_bank_of_india),4.5,2.2,2.3,0.333333
2,1,50.75,net_banking(icici_bank),2024-06-03,1,cash_on_delivery,6.0,1.6,4.4,0.800000
3,3,200.00,net_banking(kotak_bank),2024-06-04,1,net_banking(axis_bank),7.5,1.9,5.6,1.000000
4,4,75.99,credit_or_debit(rupay),2024-06-05,0,credit_or_debit(american express),4.0,1.8,2.2,0.888889
...,...,...,...,...,...,...,...,...,...,...
82,4,230.50,credit_or_debit(american express),2024-08-22,1,credit_or_debit(american express),6.0,2.5,3.5,0.500000
83,12,95.99,credit_or_debit(rupay),2024-08-23,1,credit_or_debit(rupay),4.0,1.8,2.2,0.888889
84,5,240.25,net_banking(hdfc_bank),2024-08-24,0,net_banking(hdfc_bank),6.5,1.8,4.7,0.200000
85,1,180.50,credit_or_debit(maestro),2024-08-25,1,cash_on_delivery,3.5,1.7,1.8,0.444444


In [580]:
merged_df['payment_method_encoded'] = le.fit_transform(merged_df['payment_method'])
merged_df['frequently_used_payment_method_encoded'] = le.fit_transform(merged_df['frequently_used_payment_method'])
merged_df

,user_id,transaction_amount,payment_method,date,success/failure,frequently_used_payment_method,cashbacks,charges,net_benefit,success_rate,payment_method_encoded,frequently_used_payment_method_encoded
0,1,150.25,credit_or_debit(visa),2024-06-01,1,cash_on_delivery,5.0,2.0,3.0,0.777778,7,1
1,2,300.50,credit_or_debit(mastercard),2024-06-02,0,net_banking(state_bank_of_india),4.5,2.2,2.3,0.333333,5,9
2,1,50.75,net_banking(icici_bank),2024-06-03,1,cash_on_delivery,6.0,1.6,4.4,0.800000,10,1
3,3,200.00,net_banking(kotak_bank),2024-06-04,1,net_banking(axis_bank),7.5,1.9,5.6,1.000000,11,7
4,4,75.99,credit_or_debit(rupay),2024-06-05,0,credit_or_debit(american express),4.0,1.8,2.2,0.888889,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...
82,4,230.50,credit_or_debit(american express),2024-08-22,1,credit_or_debit(american express),6.0,2.5,3.5,0.500000,3,2
83,12,95.99,credit_or_debit(rupay),2024-08-23,1,credit_or_debit(rupay),4.0,1.8,2.2,0.888889,6,5
84,5,240.25,net_banking(hdfc_bank),2024-08-24,0,net_banking(hdfc_bank),6.5,1.8,4.7,0.200000,9,8
85,1,180.50,credit_or_debit(maestro),2024-08-25,1,cash_on_delivery,3.5,1.7,1.8,0.444444,4,1


In [581]:
pm_df=merged_df[['payment_method','payment_method_encoded','net_benefit','success_rate']]
pm_df.drop_duplicates(inplace=True)
pm_df

C:\Users\mswat\AppData\Local\Temp\ipykernel_14992\2557712912.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pm_df.drop_duplicates(inplace=True)


,payment_method,payment_method_encoded,net_benefit,success_rate
0,credit_or_debit(visa),7,3.0,0.777778
1,credit_or_debit(mastercard),5,2.3,0.333333
2,net_banking(icici_bank),10,4.4,0.800000
3,net_banking(kotak_bank),11,5.6,1.000000
4,credit_or_debit(rupay),6,2.2,0.888889
5,cash_on_delivery,2,0.0,1.000000
6,credit_or_debit(american express),3,3.5,0.500000
7,EMI,0,7.0,0.000000
8,UPI,1,2.0,1.000000
9,net_banking(axis_bank),8,5.5,1.000000


In [582]:
features = ['transaction_amount', 'net_benefit', 'success_rate', 'frequently_used_payment_method_encoded']
X = merged_df[features]
y = merged_df['payment_method_encoded']

In [583]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [584]:
features2 = ['transaction_amount', 'net_benefit', 'success_rate']
X2 = merged_df[features2]
y2 = merged_df['payment_method_encoded']

In [585]:
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

model2 = RandomForestClassifier(n_estimators=100, random_state=42)
model2.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [586]:
def get_frequently_used_payment_method(user_id, transaction_df):
    user_transactions = transaction_df[transaction_df['user_id'] == user_id]
    user_transactions=user_transactions.sort_values(by=list(user_transactions.columns), ascending=[False] * len(user_transactions.columns))
    if not user_transactions.empty:
        return user_transactions['frequently_used_payment_method'].mode()[0]
    else:
        return None

In [587]:
transaction = {
    'user_id': 4,
    'transaction_amount': 150,
    'date': '2024-06-17',
    'success/failure': 'success'
}

In [588]:
transaction

{'user_id': 4,
 'transaction_amount': 150,
 'date': '2024-06-17',
 'success/failure': 'success'}

In [589]:
user_id = transaction['user_id']
frequently_used_payment_method = get_frequently_used_payment_method(user_id, transaction_df)
print(frequently_used_payment_method)

credit_or_debit(american express)


In [590]:
transaction = pd.DataFrame([transaction])
transaction['date'] = pd.to_datetime(transaction['date'])
transaction['success/failure'] = np.where(transaction['success/failure'] == 'success', 1, 0)
transaction

,user_id,transaction_amount,date,success/failure
0,4,150,2024-06-17,1


In [591]:
if frequently_used_payment_method!=None:
    transaction['frequently_used_payment_method'] = frequently_used_payment_method
transaction

,user_id,transaction_amount,date,success/failure,frequently_used_payment_method
0,4,150,2024-06-17,1,credit_or_debit(american express)


In [592]:
current_offers = offers_df[(offers_df['start_date'] <= transaction['date'].values[0]) & (offers_df['end_date'] >= transaction['date'].values[0])]
current_offers

,payment_method,start_date,end_date,cashbacks,charges
0,credit_or_debit(visa),2024-01-01,2024-12-31,5.0,2.0
1,credit_or_debit(mastercard),2024-01-01,2024-12-31,4.5,2.2
2,credit_or_debit(rupay),2024-01-01,2024-12-31,4.0,1.8
3,credit_or_debit(american express),2024-01-01,2024-12-31,6.0,2.5
4,credit_or_debit(maestro),2024-01-01,2024-12-31,3.5,1.7
5,net_banking(axis_bank),2024-01-01,2024-12-31,7.0,1.5
6,net_banking(hdfc_bank),2024-01-01,2024-12-31,6.5,1.8
7,net_banking(kotak_bank),2024-01-01,2024-12-31,7.5,1.9
8,net_banking(icici_bank),2024-01-01,2024-12-31,6.0,1.6
9,net_banking(state_bank_of_india),2024-01-01,2024-12-31,8.0,2.0


In [593]:
if frequently_used_payment_method:
    transaction['frequently_used_payment_method_encoded'] = le.transform([transaction['frequently_used_payment_method'].values[0]])
transaction

,user_id,transaction_amount,date,success/failure,frequently_used_payment_method,frequently_used_payment_method_encoded
0,4,150,2024-06-17,1,credit_or_debit(american express),2


In [594]:
repeated_row = pd.concat([transaction] * len(current_offers), ignore_index=True)
transaction = pd.concat([current_offers, repeated_row], axis=1)
transaction

,payment_method,start_date,end_date,cashbacks,charges,user_id,transaction_amount,date,success/failure,frequently_used_payment_method,frequently_used_payment_method_encoded
0,credit_or_debit(visa),2024-01-01,2024-12-31,5.0,2.0,4,150,2024-06-17,1,credit_or_debit(american express),2
1,credit_or_debit(mastercard),2024-01-01,2024-12-31,4.5,2.2,4,150,2024-06-17,1,credit_or_debit(american express),2
2,credit_or_debit(rupay),2024-01-01,2024-12-31,4.0,1.8,4,150,2024-06-17,1,credit_or_debit(american express),2
3,credit_or_debit(american express),2024-01-01,2024-12-31,6.0,2.5,4,150,2024-06-17,1,credit_or_debit(american express),2
4,credit_or_debit(maestro),2024-01-01,2024-12-31,3.5,1.7,4,150,2024-06-17,1,credit_or_debit(american express),2
5,net_banking(axis_bank),2024-01-01,2024-12-31,7.0,1.5,4,150,2024-06-17,1,credit_or_debit(american express),2
6,net_banking(hdfc_bank),2024-01-01,2024-12-31,6.5,1.8,4,150,2024-06-17,1,credit_or_debit(american express),2
7,net_banking(kotak_bank),2024-01-01,2024-12-31,7.5,1.9,4,150,2024-06-17,1,credit_or_debit(american express),2
8,net_banking(icici_bank),2024-01-01,2024-12-31,6.0,1.6,4,150,2024-06-17,1,credit_or_debit(american express),2
9,net_banking(state_bank_of_india),2024-01-01,2024-12-31,8.0,2.0,4,150,2024-06-17,1,credit_or_debit(american express),2


In [595]:
transaction['net_benefit'] = transaction['cashbacks'] - transaction['charges']
transaction

,payment_method,start_date,end_date,cashbacks,charges,user_id,transaction_amount,date,success/failure,frequently_used_payment_method,frequently_used_payment_method_encoded,net_benefit
0,credit_or_debit(visa),2024-01-01,2024-12-31,5.0,2.0,4,150,2024-06-17,1,credit_or_debit(american express),2,3.0
1,credit_or_debit(mastercard),2024-01-01,2024-12-31,4.5,2.2,4,150,2024-06-17,1,credit_or_debit(american express),2,2.3
2,credit_or_debit(rupay),2024-01-01,2024-12-31,4.0,1.8,4,150,2024-06-17,1,credit_or_debit(american express),2,2.2
3,credit_or_debit(american express),2024-01-01,2024-12-31,6.0,2.5,4,150,2024-06-17,1,credit_or_debit(american express),2,3.5
4,credit_or_debit(maestro),2024-01-01,2024-12-31,3.5,1.7,4,150,2024-06-17,1,credit_or_debit(american express),2,1.8
5,net_banking(axis_bank),2024-01-01,2024-12-31,7.0,1.5,4,150,2024-06-17,1,credit_or_debit(american express),2,5.5
6,net_banking(hdfc_bank),2024-01-01,2024-12-31,6.5,1.8,4,150,2024-06-17,1,credit_or_debit(american express),2,4.7
7,net_banking(kotak_bank),2024-01-01,2024-12-31,7.5,1.9,4,150,2024-06-17,1,credit_or_debit(american express),2,5.6
8,net_banking(icici_bank),2024-01-01,2024-12-31,6.0,1.6,4,150,2024-06-17,1,credit_or_debit(american express),2,4.4
9,net_banking(state_bank_of_india),2024-01-01,2024-12-31,8.0,2.0,4,150,2024-06-17,1,credit_or_debit(american express),2,6.0


In [596]:
transaction['success_rate'] = transaction.groupby(['payment_method'])['success/failure'].transform(lambda x: x.sum() / x.count())
transaction

,payment_method,start_date,end_date,cashbacks,charges,user_id,transaction_amount,date,success/failure,frequently_used_payment_method,frequently_used_payment_method_encoded,net_benefit,success_rate
0,credit_or_debit(visa),2024-01-01,2024-12-31,5.0,2.0,4,150,2024-06-17,1,credit_or_debit(american express),2,3.0,1.0
1,credit_or_debit(mastercard),2024-01-01,2024-12-31,4.5,2.2,4,150,2024-06-17,1,credit_or_debit(american express),2,2.3,1.0
2,credit_or_debit(rupay),2024-01-01,2024-12-31,4.0,1.8,4,150,2024-06-17,1,credit_or_debit(american express),2,2.2,1.0
3,credit_or_debit(american express),2024-01-01,2024-12-31,6.0,2.5,4,150,2024-06-17,1,credit_or_debit(american express),2,3.5,1.0
4,credit_or_debit(maestro),2024-01-01,2024-12-31,3.5,1.7,4,150,2024-06-17,1,credit_or_debit(american express),2,1.8,1.0
5,net_banking(axis_bank),2024-01-01,2024-12-31,7.0,1.5,4,150,2024-06-17,1,credit_or_debit(american express),2,5.5,1.0
6,net_banking(hdfc_bank),2024-01-01,2024-12-31,6.5,1.8,4,150,2024-06-17,1,credit_or_debit(american express),2,4.7,1.0
7,net_banking(kotak_bank),2024-01-01,2024-12-31,7.5,1.9,4,150,2024-06-17,1,credit_or_debit(american express),2,5.6,1.0
8,net_banking(icici_bank),2024-01-01,2024-12-31,6.0,1.6,4,150,2024-06-17,1,credit_or_debit(american express),2,4.4,1.0
9,net_banking(state_bank_of_india),2024-01-01,2024-12-31,8.0,2.0,4,150,2024-06-17,1,credit_or_debit(american express),2,6.0,1.0


In [597]:
if frequently_used_payment_method==None:
    features = ['transaction_amount', 'net_benefit', 'success_rate']
feature_values = transaction[features].values

In [598]:
if frequently_used_payment_method!=None:
    predicted_payment_method_encoded = model.predict(feature_values)
else:
    predicted_payment_method_encoded = model2.predict(feature_values)
        

C:\Users\mswat\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [599]:
predicted_payment_method_encoded

array([ 7,  6,  6,  3,  1,  8,  8, 11, 10, 12,  1, 12,  2])

In [600]:
pm_df

,payment_method,payment_method_encoded,net_benefit,success_rate
0,credit_or_debit(visa),7,3.0,0.777778
1,credit_or_debit(mastercard),5,2.3,0.333333
2,net_banking(icici_bank),10,4.4,0.800000
3,net_banking(kotak_bank),11,5.6,1.000000
4,credit_or_debit(rupay),6,2.2,0.888889
5,cash_on_delivery,2,0.0,1.000000
6,credit_or_debit(american express),3,3.5,0.500000
7,EMI,0,7.0,0.000000
8,UPI,1,2.0,1.000000
9,net_banking(axis_bank),8,5.5,1.000000


In [601]:
pm_df.columns

Index(['payment_method', 'payment_method_encoded', 'net_benefit',
       'success_rate'],
      dtype='object')

In [602]:
filtered_df = pm_df[pm_df['payment_method_encoded'].isin(predicted_payment_method_encoded)]
filtered_df

,payment_method,payment_method_encoded,net_benefit,success_rate
0,credit_or_debit(visa),7,3.0,0.777778
2,net_banking(icici_bank),10,4.4,0.800000
3,net_banking(kotak_bank),11,5.6,1.000000
4,credit_or_debit(rupay),6,2.2,0.888889
5,cash_on_delivery,2,0.0,1.000000
6,credit_or_debit(american express),3,3.5,0.500000
8,UPI,1,2.0,1.000000
9,net_banking(axis_bank),8,5.5,1.000000
12,net_banking(state_bank_of_india),12,6.0,1.000000


In [603]:
sorted_filtered_df = filtered_df.set_index('payment_method_encoded').loc[predicted_payment_method_encoded].reset_index()
pred_df=sorted_filtered_df
pred_df

,payment_method_encoded,payment_method,net_benefit,success_rate
0,7,credit_or_debit(visa),3.0,0.777778
1,6,credit_or_debit(rupay),2.2,0.888889
2,6,credit_or_debit(rupay),2.2,0.888889
3,3,credit_or_debit(american express),3.5,0.500000
4,1,UPI,2.0,1.000000
5,8,net_banking(axis_bank),5.5,1.000000
6,8,net_banking(axis_bank),5.5,1.000000
7,11,net_banking(kotak_bank),5.6,1.000000
8,10,net_banking(icici_bank),4.4,0.800000
9,12,net_banking(state_bank_of_india),6.0,1.000000


In [604]:
predicted_payment_method_encoded

array([ 7,  6,  6,  3,  1,  8,  8, 11, 10, 12,  1, 12,  2])

In [605]:
payment_method_counts = pred_df['payment_method'].value_counts()
payment_method_counts

payment_method
credit_or_debit(rupay)               2
UPI                                  2
net_banking(axis_bank)               2
net_banking(state_bank_of_india)     2
credit_or_debit(visa)                1
credit_or_debit(american express)    1
net_banking(kotak_bank)              1
net_banking(icici_bank)              1
cash_on_delivery                     1
Name: count, dtype: int64

In [606]:
max_frequency = payment_method_counts.max()
max_frequency

2

In [607]:
most_frequent_methods = payment_method_counts[payment_method_counts == max_frequency].index.tolist()
most_frequent_methods

['credit_or_debit(rupay)',
 'UPI',
 'net_banking(axis_bank)',
 'net_banking(state_bank_of_india)']

In [608]:
filtered_df = pred_df[pred_df['payment_method'].isin(most_frequent_methods)]
filtered_df

,payment_method_encoded,payment_method,net_benefit,success_rate
1,6,credit_or_debit(rupay),2.2,0.888889
2,6,credit_or_debit(rupay),2.2,0.888889
4,1,UPI,2.0,1.000000
5,8,net_banking(axis_bank),5.5,1.000000
6,8,net_banking(axis_bank),5.5,1.000000
9,12,net_banking(state_bank_of_india),6.0,1.000000
10,1,UPI,2.0,1.000000
11,12,net_banking(state_bank_of_india),6.0,1.000000


In [609]:
highest_success_rate = filtered_df['success_rate'].max()
highest_success_methods = filtered_df[filtered_df['success_rate'] == highest_success_rate]
highest_success_methods

,payment_method_encoded,payment_method,net_benefit,success_rate
4,1,UPI,2.0,1.0
5,8,net_banking(axis_bank),5.5,1.0
6,8,net_banking(axis_bank),5.5,1.0
9,12,net_banking(state_bank_of_india),6.0,1.0
10,1,UPI,2.0,1.0
11,12,net_banking(state_bank_of_india),6.0,1.0


In [610]:
if highest_success_methods.shape[0] > 1:
    highest_net_benefit = highest_success_methods['net_benefit'].max()
    final_method = highest_success_methods[highest_success_methods['net_benefit'] == highest_net_benefit]['payment_method'].iloc[0]
else:
    final_method = highest_success_methods['payment_method'].iloc[0]
final_method

'net_banking(state_bank_of_india)'